<p>
<font size='5' face='Georgia, Arial'>IIC-2115 Programación como herramienta para la ingeniería</font><br>
<font size='1'>Basado en: &copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
</p>

# Decoradores

Antes de definir lo que es un decorador, veamos algunas de las cosas que podemos hacer con las funciones en Python.

En Python uno puede asignar una función a una variable y luego usar esa variable igual que la función:

In [ ]:
def func_hola(nombre):
    return "Hola " + nombre

saludar = func_hola
print(saludar("Juan"))

También podemos definir funciones adentro de otras funciones:

In [ ]:
def func_hola(nombre):
    def get_message():
        return "Hola "

    res = get_message() + nombre
    return res

print(func_hola("Pedro"))

Las funciones también pueden ser pasadas como argumentos a otras funciones:

In [ ]:
def func_hola(nombre):
    return "Hola " + nombre

def llamar_func(func):
    otro_nombre = "Diego"
    return func(otro_nombre)  

print(llamar_func(func_hola))

Las funciones pueden retornar otras funciones:

In [ ]:
def compone_func_saludar():
    def get_message():
        return "¡Hola a todos!"

    return get_message

saludar = compone_func_saludar()
print(saludar())

Las funciones internas tienen acceso (sólo de lectura) a las variables (y argumentos) del scope de la función que la contiene:

In [ ]:
def compone_func_saludar(nombre):
    def get_message():
        return "¡Saludos para ti, {}!".format(nombre)
    return get_message

saludar = compone_func_saludar("Ana")
print(saludar())

Ahora el mismo ejemplo pero con algunos `print` que nos aclaran lo que ocurre cuando intentamos modificar una variable de la
función que está un nivel más arriba:

In [ ]:
def compone_func_saludar(nombre):
    aux = 2
    print(id(aux)) #la función id entrega un número identificador único por cada variable existente 
    print(aux)
    def get_message():
        print("Entrando a get_mensaje()...")
        aux = 3
        print(id(aux)) # acá el identificador retornado por id será distinto al anterior, ¿por qué?
        print(aux)
        return "¡Saludos para ti, {}!".format(nombre)
    print(id(aux))
    print(aux)
    return get_message  # en esta llamada se entra en la ejecución de get_message

print("Acá.")
saludar = compone_func_saludar("Ana")
print("Aquí.")
print(saludar())

Los decoradores nos permiten tomar una función ya implementada, agregar algún comportamiento o datos adicionales y retornar una nueva función. Podemos ver los decoradores simplemente como funciones que reciben una función `f1` cualquiera, y retornan una función `f2` distinta. Por ejemplo, si nuestro decorador se llama `dec_1`, para obtener la función modificada que queremos y asignarla a la misma función actual, simplemente escribimos `f1 = dec_1(f1)`. 

Con esto, nuestra función `f1` ahora queda con los nuevos datos y comportamientos agregados. Uno de los beneficios de los decoradores es que nos evitan la necesidad de modificar el código de la función original: así, si queremos volver a la versión original de la función, simplemente quitamos el llamado al decorador. También nos evita crear una función distinta con otro nombre. Esto sería un problema, ya que habría que modificar todos los llamados a la función que queremos cambiar para que llamen a la función nueva.

**Ejemplo:** supongamos que tenemos la siguiente implementación recursiva ineficiente de la función que retorna los números de Fibonacci.

In [ ]:
import datetime

def fib(n):
    if n == 0 or n == 1:
        return n
    else:
        return fib(n-1) + fib(n-2)

n = 36
t1 = datetime.datetime.now()
print(fib(n))
print("Tiempo de ejecución: {}".format(datetime.datetime.now()-t1))


Una implementación mucho más eficiente podría preocuparse de "memorizar" los números ya calculados en la secuencia de Fibonacci. 
Podemos usar un decorador que tome la función `fib` y le agregue una memoria y un chequeo que se preocupe de ver la existencia
del número en algún cálculo anterior:

In [ ]:
def efficient_fib(f):  # recibe una función como argumento
    data = {}
    def func(x):  # aquí se crea la función nueva que será retornada
        if x not in data:
            data[x] = f(x)  # aquí usa la función que recibió como argumento
        return data[x]
    return func
    
fib = efficient_fib(fib)  # aquí aplicamos el decorador, 
# la función fib queda "decorada" por la función "eficient_fib"
t1 = datetime.datetime.now()

print(fib(n))  # aquí vemos que seguimos usando el mismo nombre para la función, 
               # sin la necesidad de llamar a la función nueva
print("Tiempo de ejecución: {}".format(datetime.datetime.now()-t1))


## Decoradores con *azúcar sintáctico*  

Una forma más rápida y legible de decorar funciones es escribiendo el nombre del decorador arriba del encabezado de la función anteponiendo un @. Es la misma sintáxis que usamos cuando queremos crear properties. De hecho `property` es un decorador. 

In [ ]:
@efficient_fib
def fib(n):
    if n == 0 or n == 1:
        return n
    else:
        return fib(n-1) + fib(n-2)

n = 36
t1 = datetime.datetime.now()
print(fib(n))
print("Tiempo de ejecución: {}".format(datetime.datetime.now()-t1))


La sintáxis para llamar al decorador es distinta, pero la forma de definirlo es la misma.

## Decoradores con parámetros

Si queremos crear decoradores que acepten parámetros, debemos agregar un tercer nivel de funciones anidadas. 
Una forma genérica de hacerlo es la siguiente.

In [ ]:
def my_decorator_constructor(dec_parameters):
    def my_decorator(function):
        def wrapped_func(*args, **kwargs): # estos son argumentos de "function"
            # hacer algo aquí antes de aplicar la función
            # llamar a la función
            # en cualquier momento se puede usar dec_parameters para algo
            res = function(*args, **kwargs)
            # hacer algo después
            return res
        # retorna la sub función
        return wrapped_function
    return my_decorator

- La función más externa es el constructor del decorador.
- La función intermedia es el decorador.
- La función más interna es la función modificada.

Veamos un ejemplo.

In [ ]:
def revisar_tipo(tipo):
    def _revisar_tipo(funcion):
        def __revisar_tipo(*args):
            for arg in args:
                if not isinstance(arg, tipo):
                    raise TypeError("Todos los agrumentos deben ser del tipo {}".format(tipo))
                return funcion(*args)
        return __revisar_tipo
    return _revisar_tipo

En este ejemplo creamos un decorador que sirve para verificar que todos los argumentos de una función sean de un determinando tipo. Para determinar el tipo, necesitamos que el decorador pueda recibir este tipo como argumento. Para esto usamos el constructor. Veamos como usarlo

In [ ]:
decorador_str = revisar_tipo(str) # Usando el constructor, creamos el decorador

In [ ]:
def concatenar_strings(str1, str2):
    return str1 + str2

print(concatenar_strings(1, 2))
print(concatenar_strings("Hola y ", "chao"))

Este comportamiento no es el esperado de esta función. Solo queremos que permita concatenar strings. Vamos a decorarla.

In [ ]:
funcion_decorada = decorador_str(concatenar_strings) # Función decorada 

# Usamos la función decorada

funcion_decorada("Hola y ", "Chao")

Si intentamos usar la función decorada para sumar enteros, tendremos un error. Este error no ocurria cuando usábamos la función no decorada

In [ ]:
funcion_decorada(1, 2)

Esto mismo lo podemos hacer de una forma más legible si usamos el azúcar sintáctica.

In [ ]:
@revisar_tipo(int)
def sumar_enteros(a, b):
    return a + b

In [ ]:
sumar_enteros(1.0, 1.5)